In [ ]:
# Telco Customer Churn
# Imports & Configuration

import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    roc_auc_score, average_precision_score, precision_recall_fscore_support,
    classification_report, roc_curve, PrecisionRecallDisplay
)
import matplotlib.pyplot as plt
plt.close('all')

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [ ]:
# Load data
OUTPUT_DIR = Path("outputs"); OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

df = pd.read_csv("/content/Telco_Customer_Churn.csv")
print("Loaded:", df.shape, "rows x cols")
display(df.head())

Loaded: (7043, 21) rows x cols


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [ ]:
# calculate the NaNs in all columns
df.isnull().sum()

,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


In [ ]:
# Robust cleaning helpers

def churn_to_binary(col: pd.Series) -> pd.Series:
    """Map many churn variants to {0,1} safely; blanks -> NaN, then fill or drop later."""
    s = col.copy()
    s = s.replace(r'^\s*$', np.nan, regex=True)          # whitespace -> NaN
    s = s.astype(str).str.strip().str.lower()            # normalize case
    s = s.replace({'nan': np.nan, 'none': np.nan})
    s = s.replace({'y':'yes','t':'yes','true':'yes','1':'yes',
                   'n':'no', 'f':'no', 'false':'no','0':'no'})
    return s.map({'yes':1, 'no':0})

def coerce_numeric(series: pd.Series) -> pd.Series:
    """Force numeric by removing non-numeric chars; return float with NaN where bad."""
    s = series.astype(str).str.strip().str.replace(r'[^0-9.\-]', '', regex=True)
    return pd.to_numeric(s, errors='coerce')

def safe_add_binary(df: pd.DataFrame, col: str, yes_value='Yes') -> pd.Series:
    """Create binary 0/1 from Yes/No-like column; returns 0 when column missing."""
    if col not in df.columns:
        return pd.Series(0, index=df.index, dtype='int8')
    return (df[col].astype(str).str.strip().str.lower() == str(yes_value).lower()).astype('int8')


In [ ]:
# Target cleanup

assert "Churn" in df.columns, "Dataset must contain a 'Churn' column."
mapped = churn_to_binary(df["Churn"])
bad = mapped.isna().sum()
if bad:
    # Decide policy: drop bad labels (safest)
    print(f"Dropping {bad} rows with invalid Churn labels.")
    keep_mask = mapped.notna()
    df = df.loc[keep_mask].copy()
    mapped = mapped.loc[keep_mask]

df["Churn"] = mapped.astype('int8')
print("Churn counts:\n", df["Churn"].value_counts())


In [ ]:
# Numeric coercion (handles 'TotalCharges' blanks problem)

for col in ["MonthlyCharges", "TotalCharges", "tenure"]:
    if col in df.columns:
        df[col] = coerce_numeric(df[col])

# Drop rows that have no usable TotalCharges (usually new customers). Optional.
if "TotalCharges" in df.columns:
    drop_na_cnt = df["TotalCharges"].isna().sum()
    if drop_na_cnt:
        print(f"Dropping {drop_na_cnt} rows with NaN TotalCharges.")
        df = df.dropna(subset=["TotalCharges"]).copy()

In [ ]:
# Feature engineering

# Tenure-based
if "tenure" in df.columns:
    df["tenure_years"] = (df["tenure"] / 12.0).round(1)
    bins = [-0.1, 12, 24, 36, 48, 60, df["tenure"].max()]
    labels = ["0–1y","1–2y","2–3y","3–4y","4–5y","5+y"]
    df["tenure_group"] = pd.cut(df["tenure"], bins=bins, labels=labels, include_lowest=True)
else:
    df["tenure_years"] = 0.0
    df["tenure_group"] = "unknown"

# Financial indicator (avoid div/0)
if all(c in df.columns for c in ["MonthlyCharges", "TotalCharges"]):
    safe_total = df["TotalCharges"].replace(0, np.nan)
    df["monthly_to_total_ratio"] = (df["MonthlyCharges"] / safe_total).replace([np.inf, -np.inf], np.nan)
else:
    df["monthly_to_total_ratio"] = np.nan

# Service adoption
service_cols = ["OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV","StreamingMovies"]
for c in service_cols:
    df[c + "_bin"] = safe_add_binary(df, c, yes_value="Yes")
bin_cols = [c + "_bin" for c in service_cols]
df["service_count"]   = df[bin_cols].sum(axis=1).astype('int16')
df["high_engagement"] = (df["service_count"] >= 4).astype('int8')

# Billing/payment behaviour
df["paperless"] = safe_add_binary(df, "PaperlessBilling", yes_value="Yes")
df["autopay"]   = (df.get("PaymentMethod", pd.Series("", index=df.index))
                   .astype(str).str.contains("automatic", case=False, na=False)).astype('int8')

# Contract features
contract_map = {"Month-to-month":0, "One year":1, "Two year":2}
df["contract_ordinal"] = df.get("Contract", pd.Series("", index=df.index)).map(contract_map).fillna(0).astype('int8')
df["short_term_contract"] = (df.get("Contract","") == "Month-to-month").astype('int8')

# High monthly within contract
if "MonthlyCharges" in df.columns and "Contract" in df.columns:
    p80 = df.groupby("Contract", dropna=False)["MonthlyCharges"].transform(lambda s: s.quantile(0.80))
    df["high_monthly_in_contract"] = (df["MonthlyCharges"] >= p80).astype('int8')
else:
    df["high_monthly_in_contract"] = 0

# Risk interactions
df["risk_shortterm_highpay"]  = ((df["short_term_contract"]==1) & (df["high_monthly_in_contract"]==1)).astype('int8')
df["risk_lowservices_highpay"] = ((df["service_count"]<2) & (df["high_monthly_in_contract"]==1)).astype('int8')

print("Engineered features ready. Total columns:", len(df.columns))

In [ ]:
# Train/Test split

ID_COL = "customerID" if "customerID" in df.columns else None
TARGET = "Churn"

X = df.drop(columns=[TARGET])
y = df[TARGET].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=RANDOM_STATE
)
id_test = X_test[ID_COL].values if ID_COL else np.arange(len(X_test))

print("Split shapes:", X_train.shape, X_test.shape)

In [ ]:
# Preprocessing (numeric and categorical)

num_cols = X_train.select_dtypes(include=["int64","float64","int32","float32","int16","int8"]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object","category","bool"]).columns.tolist()
if ID_COL in num_cols: num_cols.remove(ID_COL)
if ID_COL in cat_cols: cat_cols.remove(ID_COL)

numeric_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer([
    ("num", numeric_tf, num_cols),
    ("cat", categorical_tf, cat_cols)
], remainder="drop")

In [ ]:
# Baseline models (fast, reliable)

logreg_pipe = Pipeline([
    ("prep", preprocess),
    ("model", LogisticRegression(max_iter=600, class_weight="balanced", random_state=RANDOM_STATE))
])

rf_pipe = Pipeline([
    ("prep", preprocess),
    ("model", RandomForestClassifier(
        n_estimators=350, class_weight="balanced",
        random_state=RANDOM_STATE, n_jobs=-1
    ))
])

logreg_pipe.fit(X_train, y_train)
rf_pipe.fit(X_train, y_train)

In [ ]:
# Evaluation helper and baseline metrics

def evaluate(pipe, name):
    probs = pipe.predict_proba(X_test)[:, 1]
    preds = (probs >= 0.5).astype(int)
    roc = roc_auc_score(y_test, probs)
    ap  = average_precision_score(y_test, probs)  # PR-AUC
    prec, rec, f1, _ = precision_recall_fscore_support(y_test, preds, average="binary", zero_division=0)
    print(f"\n{name}")
    print(f"ROC-AUC: {roc:.3f} | PR-AUC: {ap:.3f} | Precision: {prec:.3f} | Recall: {rec:.3f} | F1: {f1:.3f}")
    print(classification_report(y_test, preds, digits=3))
    return {"roc_auc": roc, "pr_auc": ap}

m1 = evaluate(logreg_pipe, "Logistic Regression (baseline)")
m2 = evaluate(rf_pipe,     "Random Forest (baseline)")

best_model, best_name = (rf_pipe, "RandomForest") if m2["pr_auc"] >= m1["pr_auc"] else (logreg_pipe, "LogisticRegression")
print("\nSelected model:", best_name)


In [ ]:
# Light tuning on the selected model (fast randomized search)

cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=RANDOM_STATE)

if best_name == "RandomForest":
    param_dist = {
        "model__n_estimators": [200, 300, 400],
        "model__max_depth": [None, 8, 12, 16],
        "model__min_samples_split": [2, 4, 6],
        "model__min_samples_leaf": [1, 2, 3],
        "model__max_features": ["sqrt", "log2", 0.5]
    }
else:
    param_dist = {"model__C": np.linspace(0.3, 2.5, 6)}

search = RandomizedSearchCV(
    best_model,
    param_distributions=param_dist,
    n_iter=10,             # keep light for MacBook Air
    scoring="roc_auc",
    cv=cv,
    n_jobs=-1,
    random_state=RANDOM_STATE,
    verbose=1,
    error_score="raise"
)
search.fit(X_train, y_train)
final_model = search.best_estimator_
print("Best params:", search.best_params_)

# Final evaluation
_ = evaluate(final_model, f"{best_name} (tuned)")

In [ ]:
# Threshold sweep and export predictions for dashboards

probs = final_model.predict_proba(X_test)[:, 1]

def sweep_thresholds(y_true, p, steps=41):
    rows = []
    for thr in np.linspace(0, 1, steps):
        pred = (p >= thr).astype(int)
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, pred, average="binary", zero_division=0)
        rows.append({"threshold": thr, "precision": precision, "recall": recall, "f1": f1})
    return pd.DataFrame(rows)

thr_df = sweep_thresholds(y_test, probs)
best_row = thr_df.sort_values("f1", ascending=False).iloc[0]
BEST_THR = float(best_row["threshold"])
print("\nBest threshold by F1:", BEST_THR, "| row:", dict(best_row))

thr_df.to_csv(OUTPUT_DIR / "threshold_sweep.csv", index=False)

risk = pd.cut(probs, [-0.01, 0.4, 0.7, 1.01], labels=["Low","Medium","High"])
preds_df = pd.DataFrame({
    "customerID": id_test,
    "churn_prob": probs,
    "predicted_label": (probs >= BEST_THR).astype(int),
    "risk_segment": risk.astype(str)
})
preds_df.to_csv(OUTPUT_DIR / "predictions.csv", index=False)
display(preds_df.head())

# PR & ROC plots
disp = PrecisionRecallDisplay.from_predictions(y_test, probs)
plt.title("Precision-Recall Curve");
plt.savefig(OUTPUT_DIR / "pr_curve.png", dpi=150, bbox_inches="tight"); plt.close()

fpr, tpr, _ = roc_curve(y_test, probs)
plt.figure(); plt.plot(fpr, tpr, label=f"ROC-AUC={roc_auc_score(y_test, probs):.2f}")
plt.plot([0,1],[0,1],'--'); plt.xlabel("FPR"); plt.ylabel("TPR"); plt.legend()
plt.savefig(OUTPUT_DIR / "roc_curve.png", dpi=150, bbox_inches="tight"); plt.close()

In [ ]:
# Save summary metrics

import json
summary = {
    "model": best_name,
    "best_params": search.best_params_,
    "roc_auc": float(roc_auc_score(y_test, probs)),
    "pr_auc": float(average_precision_score(y_test, probs)),
    "best_threshold": BEST_THR,
    "f1_at_best_threshold": float(best_row["f1"]),
    "precision_at_best_threshold": float(best_row["precision"]),
    "recall_at_best_threshold": float(best_row["recall"]),
    "n_features_train": int(X_train.shape[1]),
    "n_rows_train": int(X_train.shape[0])
}
with open(OUTPUT_DIR / "metrics.json", "w") as f:
    json.dump(summary, f, indent=2)

print("\nArtifacts saved to:", OUTPUT_DIR.resolve())
print(" - predictions.csv")
print(" - threshold_sweep.csv")
print(" - pr_curve.png, roc_curve.png")
print(" - metrics.json")


Loaded: (7043, 21) rows x cols


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


Churn counts:
 Churn
0    5174
1    1869
Name: count, dtype: int64
Dropping 11 rows with NaN TotalCharges.
Engineered features ready. Total columns: 39
Split shapes: (5625, 38) (1407, 38)

Logistic Regression (baseline)
ROC-AUC: 0.839 | PR-AUC: 0.655 | Precision: 0.493 | Recall: 0.813 | F1: 0.614
              precision    recall  f1-score   support

           0      0.911     0.697     0.790      1033
           1      0.493     0.813     0.614       374

    accuracy                          0.728      1407
   macro avg      0.702     0.755     0.702      1407
weighted avg      0.800     0.728     0.743      1407


Random Forest (baseline)
ROC-AUC: 0.817 | PR-AUC: 0.596 | Precision: 0.610 | Recall: 0.468 | F1: 0.530
              precision    recall  f1-score   support

           0      0.822     0.892     0.856      1033
           1      0.610     0.468     0.530       374

    accuracy                          0.779      1407
   macro avg      0.716     0.680     0.693      1407

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best params: {'model__C': np.float64(2.5)}

LogisticRegression (tuned)
ROC-AUC: 0.839 | PR-AUC: 0.655 | Precision: 0.491 | Recall: 0.813 | F1: 0.612
              precision    recall  f1-score   support

           0      0.911     0.695     0.789      1033
           1      0.491     0.813     0.612       374

    accuracy                          0.726      1407
   macro avg      0.701     0.754     0.700      1407
weighted avg      0.800     0.726     0.742      1407


Best threshold by F1: 0.625 | row: {'threshold': np.float64(0.625), 'precision': np.float64(0.5698689956331878), 'recall': np.float64(0.6978609625668449), 'f1': np.float64(0.6274038461538461)}


...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_9['risk_segment'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_9, x='churn_prob', y='risk_segment', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_10['customerID'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_10, x='predicted_label', y='customerID', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_11['risk_segment'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_11, x='predicted_label', y='risk_segment', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
#Estimating a realistic Churn Reduction Percentage using Model's Recall and Precision

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, recall_score

# Load the dataset
df = pd.read_csv("/content/Telco_Customer_Churn.csv")

# Encode categorical variables
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])

# Features and target
X = df.drop(columns=["Churn"])
y = df["Churn"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predictions
y_pred_proba = model.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba >= 0.5).astype(int)

# Model metrics
roc_auc = roc_auc_score(y_test, y_pred_proba)
recall = recall_score(y_test, y_pred)

# Business simulation
# Assume retention campaign targets customers with churn probability > 0.7
# Assume campaign can save 35% of those customers
high_risk_customers = (y_pred_proba > 0.7) & (y_test == 1)
num_high_risk = high_risk_customers.sum()
saved_customers = int(num_high_risk * 0.35)

# Current churn rate
current_churn_rate = y_test.mean()
# New churn rate after campaign
new_churn_rate = (y_test.sum() - saved_customers) / len(y_test)

# Reduction percentage
reduction_percentage = ((current_churn_rate - new_churn_rate) / current_churn_rate) * 100

roc_auc, recall, current_churn_rate, new_churn_rate, reduction_percentage


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(np.float64(0.8602792758288738),
 0.5764075067024129,
 np.float64(0.26472675656493966),
 np.float64(0.24769339957416608),
 np.float64(6.434316353887391))

In [ ]:
#Plotting the Visuals

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the Telco dataset
df = pd.read_csv("/content/Telco_Customer_Churn.csv")

# Ensure 'Churn' is binary
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

# Churn probability distribution (simulate predicted probabilities for illustration)
import numpy as np
np.random.seed(42)
df['Churn_Probability'] = np.where(df['Churn'] == 1,
                                   np.random.uniform(0.5, 1, df.shape[0]),
                                   np.random.uniform(0, 0.5, df.shape[0]))

# Plot Churn Probability Distribution
plt.figure(figsize=(8,5))
sns.histplot(data=df, x='Churn_Probability', hue='Churn', kde=True, bins=20, palette='coolwarm')
plt.title("Churn Probability Distribution")
plt.xlabel("Predicted Probability of Churn")
plt.ylabel("Number of Customers")
plt.tight_layout()
plt.savefig("churn_probability_distribution.png")
plt.close()

# Customer segmentation based on tenure and monthly charges
df['Tenure_Group'] = pd.cut(df['tenure'], bins=[0, 12, 24, 48, 72], labels=['0-1 yr', '1-2 yrs', '2-4 yrs', '4-6 yrs'])

plt.figure(figsize=(8,5))
sns.scatterplot(data=df, x='tenure', y='MonthlyCharges', hue='Churn', palette='coolwarm', alpha=0.6)
plt.title("Customer Segmentation by Tenure & Monthly Charges")
plt.xlabel("Tenure (Months)")
plt.ylabel("Monthly Charges ($)")
plt.tight_layout()
plt.savefig("customer_segmentation.png")
plt.close()

# Impact simulation: showing potential churn reduction if high-risk customers are retained
threshold = 0.7
high_risk_customers = df[df['Churn_Probability'] > threshold].shape[0]
potential_saved = int(high_risk_customers * 0.5)  # Assume we save 50% of them
current_churn_count = df['Churn'].sum()
new_churn_count = current_churn_count - potential_saved
reduction_percentage = (potential_saved / current_churn_count) * 100

impact_text = f"High-risk customers (>70% churn probability): {high_risk_customers}\n" \
              f"Potentially saved (50% retention): {potential_saved}\n" \
              f"Reduction in churn: {reduction_percentage:.2f}%"

# Create a bar chart to visualize
plt.figure(figsize=(6,4))
plt.bar(['Current Churn', 'After Retention Program'], [current_churn_count, new_churn_count], color=['red','green'])
plt.ylabel("Number of Churned Customers")
plt.title("Impact Simulation: Churn Reduction Potential")
plt.tight_layout()
plt.savefig("impact_simulation.png")
plt.close()
